In [2]:
import os

while not os.getcwd().endswith("hackathon"):
    os.chdir("..")
    print(f"Now in {os.getcwd()}")

Now in c:\Users\GiovanniGiacometti\Desktop\hackathon\notebooks
Now in c:\Users\GiovanniGiacometti\Desktop\hackathon


In [3]:
MENU_FOLDER = "competition_data/Menu/"

In [4]:
file_list = os.listdir(MENU_FOLDER)

In [5]:
file_paths = []
for dir_entry in os.scandir(MENU_FOLDER):
    file_paths.append(dir_entry.path)

In [6]:
import pymupdf4llm

md_text = pymupdf4llm.to_markdown(file_paths[11])

Processing competition_data/Menu/L Essenza Cosmica.pdf...
[                                        ] (0/6=====[======                                  ] (1/6======[=============                           ] (2/======[====================                    ] (3/6=====[==========================              ] (4/6======[=================================       ] (5/======[========================================] (6/6]


In [7]:
from hackathon.utils.settings.settings_provider import SettingsProvider
import warnings
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

with warnings.catch_warnings(action="ignore"):
    from langchain_ibm import ChatWatsonx
import dotenv

dotenv.load_dotenv();

In [7]:
# settings_provider = SettingsProvider()

# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": False}

# vectorstore = Chroma(
#     persist_directory="data/metadata-tests",
#     embedding_function=HuggingFaceEmbeddings(
#             model_name=settings_provider.get_embeddings_model_name(),
#             model_kwargs=model_kwargs,
#             encode_kwargs=encode_kwargs,
#         ),
# )

In [8]:
settings_provider = SettingsProvider()

llm = ChatWatsonx(
    model_id=settings_provider.get_ibm_model_name(),  # type: ignore
    url=settings_provider.get_ibm_endpoint_url(),  # type: ignore
    project_id=settings_provider.get_ibm_project_id(),  # type: ignore
)

In [9]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate


headers_to_split_on = [("#", "header_1"), ("##", "dish_name")]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_text)
md_header_splits

[Document(metadata={}, page_content='**Ristorante "L\'Essenza Cosmica"**  \nChef Alessandro Stellanova  \nSu Pandora, dove il confine tra la natura rigogliosa e l\'infinito universo si annulla, sorge "L\'Essenza\nCosmica", un ristorante che sfida le leggi conosciute dell\'esperienza culinaria. Qui, lo Chef Alessandro\nStellanova non si limita a cucinare; crea simbiosi tra l\'arte del gusto e l\'energia vivente del pianeta.  \nIl viaggio culinario di Stellanova ebbe origine in un piccolo angolo del suo mondo natale, dove comprese che\nla trasformazione degli ingredienti rispecchiava le strutture essenziali dell\'universo stesso. Questa intuizione\nlo ha portato a perfezionare una straordinaria padronanza della Psionica (Livello V), che utilizza per ideare\nricette che toccano l\'essenza stessa dell\'esistenza. I visitatori descrivono piatti che evocano memorie di vite\nnon vissute e sapori che risuonano in dimensioni intricate.  \nNel suo laboratorio culinario, la Quantistica (Livello 3

In [10]:
from enum import Enum
from pydantic import BaseModel, Field, model_validator


class DishMetadata(BaseModel):
    dish_techniques: list | None = Field(
        description="Techniques involved in making the dish", default_factory=list
    )
    dish_ingredients: list | None = Field(
        description="Ingredients used in the dish", default_factory=list
    )

    @model_validator(mode="after")
    def max_3_techniques(self):
        if len(self.dish_ingredients) > 3:
            raise ValueError("Dish metadata should have at most 3 techniques")
        if len(self.dish_techniques) > 3:
            raise ValueError("Dish metadata should have at most 3 ingredients")
        return self


dish_metadata_keys = list(DishMetadata.model_fields.keys())

In [23]:
from langchain_core.utils.function_calling import convert_to_openai_tool

In [24]:
convert_to_openai_tool(DishMetadata)

{'type': 'function',
 'function': {'name': 'DishMetadata',
  'description': '',
  'parameters': {'properties': {'dish_techniques': {'anyOf': [{'items': {},
       'type': 'array'},
      {'type': 'null'}],
     'description': 'Techniques involved in making the dish'},
    'dish_ingredients': {'anyOf': [{'items': {}, 'type': 'array'},
      {'type': 'null'}],
     'description': 'Ingredients used in the dish'}},
   'type': 'object'}}}

In [25]:
llm_with_struct = llm.with_structured_output(convert_to_openai_tool(DishMetadata))

In [26]:
prompt = """
You are an expert metadata extractor, focused on cuisine. These are the metadata you need to find: {metadata}.
If the metadata is a list, include at most 3 items. Look for the
most relevant metadata values: they usually have strange names and start with capital letters.
"""

In [27]:
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        (
            "human",
            "Extract the metadata from the following document {document}",
        ),
    ]
)

In [28]:
metadata_extract_chain = answer_prompt | llm_with_struct

In [29]:
# from IPython.display import display, Markdown

# display(Markdown(md_header_splits[1].page_content))

In [30]:
t = metadata_extract_chain.invoke(
    {
        "document": md_header_splits[1].page_content,
        "metadata": dish_metadata_keys,
    }
)

OutputParserException: Function DishMetadata arguments:

{"dish_techniques": ["Marinatura Temporale Sincronizzata", "Grigliatura Elettro-Molecolare a Spaziatura Variabile", "Taglio a Risonanza Sonica Rigenerativa", "Cottura con radiazione dorata di una stella lontana", "Infusione di Nettare di Sirena e Salsa Szechuan", "Addensamento con Amido di Stellarion", "Salsa vellutata dal gusto ipnotico e rigenerante", "Echeggiamento delle maree dell'universo", "Contrasto netto e vibrante nel piatto", "Esplorazione di dimensioni e possibilità", "Sapori che continuano a evolversi e intrecciarsi sul tuo palato", "Un'esperienza gastronomica che solo 'L'Essenza Cosmica' può offrire", "Creazione interstellare", "Stratificazione di sapori", "Invito il palato a esplorare ogni angolo di questa creazione interstellare", "Sigillatura dei succhi e degli aromi attraverso un campo energetico calibrato", "Creazione di una salsa vellutata dal gusto ipnotico e rigenerante", "Esecuzione di un Taglio a Risonanza 

In [ ]:
for md_header_split in md_header_splits:
    vectorstore.add_texts(
        texts=[md_header_split.page_content],
        metadatas=md_header_split.metadata,
    )